In [345]:
from transformers import (
    AutoTokenizer, AutoModelForMaskedLM, 
)

# import nlpaug.augmenter.char as nac
# import nlpaug.augmenter.word as naw
# import nlpaug.augmenter.sentence as nas
# import nlpaug.flow as nafc
# from nlpaug.util import Action

# from googletrans import Translator
# import translators as ts

import re, math, random, json
from sklearn.model_selection import StratifiedKFold
import pandas as pd
from tqdm import tqdm
# import demoji

from cleantext import clean
# from pykospacing import Spacing
# from hanspell import spell_checker

In [346]:
# train_json = './dataset//nikluge-sa-2022-train.jsonl'
# train_raw = pd.read_json(train_json, lines=True)
# train_raw.annotation = train_raw.annotation.apply(lambda x: str(x))
# train_raw = train_raw[['sentence_form', 'annotation']]
# dup = train_raw.duplicated(keep=False)
# train_raw[dup]

In [347]:
# dev_json = './dataset/nikluge-sa-2022-dev.jsonl'
# dev_raw = pd.read_json(dev_json, lines=True)
# dev_raw.annotation = dev_raw.annotation.apply(lambda x: str(x))
# dev_raw = dev_raw[['sentence_form', 'annotation']]
# dup = dev_raw.duplicated(keep=False)
# dev_raw[dup]

In [531]:
train_json = './dataset/nikluge-sa-2022-train.jsonl'
dev_json = './dataset/nikluge-sa-2022-dev.jsonl'
test_json = './dataset/nikluge-sa-2022-test.jsonl'

In [532]:
train = pd.read_json(train_json, lines=True)
dev = pd.read_json(dev_json, lines=True)
test = pd.read_json(test_json, lines=True)

In [533]:
# train.iloc[2319]
train = train.drop(2319)
# dev.iloc[1692]
dev = dev.drop(1692)

# Preprocess

## Cleansing

### Before

In [534]:
for el in train.sample(n=15, frac=None, replace=False, weights=None, random_state=None, axis=None, ignore_index=False).sentence_form:
    print(el)

그냥 모기장만 설치했을때도 예뻤지만 보태니컬 가랜드랑 같이 해놓으니 더 만족도 UP
내가 좋아하는 아기옷브랜드 #에뜨와
저처럼 #건성피부 라면 더욱 만족을 느끼실 것 같은 #수분력 도 짱짱!
놀라운 자연스러운 #보습 과 #광
이뽀+_+
밤에 감으면 좋은 샴푸래요
대박! ㅋㅋ
믿고 쓰는 #뷰티챌린지 베스트템
미니냉장고가방으로 불리는 이유가 있어
운동 끝나고 승모근 쪽 당길때도 마사지하기에도 좋고!👏🏻👏🏻
60초면 흡수 끝!!
100% 무기자외선차단 성분으로 더욱 순하게 향도 순하니 좋다!
그렇다고 가격이 그렇게 비싸냐하면 절대 그렇지 않다.
물을 흡수하는 구조가 아니라서 오염에도 걱정 NO
부드럽고 촉촉하게 클렌징 하나로 홈케어 마사지로 깨끗하게 클렌징


In [535]:
def preprocess(text):
    text = re.sub(r'&.+&', '<숨김>', text)
    text = re.sub(r'[^A-Za-z0-9가-힣]', ' ', text)
    
    text = clean(
        text,
        fix_unicode=False,
        to_ascii=False,
        lower=True,
        normalize_whitespace=True,
        no_line_breaks=False,
        strip_lines=True,
        keep_two_line_breaks=False,
        no_urls=False,
        no_emails=False,
        no_phone_numbers=False,
        no_numbers=True,
        no_digits=True,
        no_currency_symbols=False,
        no_punct=True,
        no_emoji=True,
        replace_with_url="<URL>",
        replace_with_email="<EMAIL>",
        replace_with_phone_number="<PHONE>",
        replace_with_number="0",
        replace_with_digit="0",
        replace_with_currency_symbol="<CUR>",
        replace_with_punct="",
        lang="en",
    )
    
    # sent = demoji.replace_with_desc(string=sent, sep= " ")
    # sent = re.sub(r'\s+', ' ', sent)
    # sent = re.sub(r'^#$', '', sent)
    
    # text = re.sub(r'[A-Za-z]+', 'alphabet', text)
    # text = re.sub(r'[`~$^+=|><]', '', text)
    # text = re.sub(r'[ㄱ-ㅎ|ㅏ-ㅣ]', '', text)
    
    # text = spacing(text)
    # text = spell_checker.check(text).checked
    return text.upper().strip()

In [536]:
train.sentence_form = train.sentence_form.apply(preprocess)
dev.sentence_form = dev.sentence_form.apply(preprocess)
test.sentence_form = test.sentence_form.apply(preprocess)
total = pd.concat([train, dev, test])

Test

In [537]:
case = total.sentence_form.str.contains('r[^A-Za-z0-9가-힣\s]+', case=False, flags=0, na=None, regex=True)
for e in total[case].sentence_form:
    print(e)

### After

In [555]:
for el in total.sample(n=25, frac=None, replace=False, weights=None, random_state=None, axis=None, ignore_index=False).sentence_form:
    print(el)

인케어가 건조한 모발을 촉촉하게 채워주고 아웃케어가 모발표면을 코팅하여 매끄럽고 단단하게
브랜드 만족도
나는 피부가 예민한 편이라 순한 제품이 잘 맞는데 유칼립투스버블샴푸앤바스 를 폼클렌징으로 사용하면서부터 뾰루지도 잘 안나고 만족해서 계속 쓰는중
앞으로 쭉쭉 듀이셀 사용해서 피부만은 나이먹지 않게 관리해야지
베베누보 신상품 침낭형 낮잠이불로 요즘 재워보고 있는데 패드가 우선 도톰하고 일체형또는 분리할수있어 편합니다
0회용 제품이라 나에게 얼마나 사용하는지 확인할 수 있어 믿음이 확 가는 제품이에요
성능과 더불어 전력 효율까지 향상되었고 램도 DDR0 0GB 0000MHZ 램으로 전력 효율과 성능이 향상된 제품입니다
피부밸런스를 맞춰주는 홀플랜트이펙트 성분이 원래의 건강한 피부로 리턴
구성이 아주 빵빵합니당
여기서 건져갈게 이거밖에
스타일00 00대여자선물추천하는 스와로브스키 잉크벨벳틴트 중 발렌타인 키스 레드를 바르고 대표사진 찰칵
크림쓰고 엄청 만족스러웠는데 치약도 맘에 쏙
다크닝없이 깨끗하게 피부를 커버를 해줘 내가 바르는 비비크림
붉은기 많고 푸석했던 피부가 자기전에 간단히 슬리핑팩 발라주고 나니 다음날 아침에 촉촉하고 피부도 맑아진게 보여서 넘 좋더라구요
원래 쓰던 노트북도 엘지라서 이번에도 엘지로 가고 싶었고 그렇게 총 네가지 모델로 좁혀졌다
0 IN 0 이라 완전 편하다
세번째 엄청난 가성비
피부가 숨 쉬는 여름에 최적인 여름용기저귀
이 인증은 동물성 원료 및 유래 원료를 포함하지 않으며 동물 실험을 하지 않았다는 의미로 더욱 안심하고 사용할 수 있느 화장품이에요
가성비좋은휴지 로 잘풀리는집휴지 추천해요
피지는 잡아주고 수분은 지켜주는 매직펜
기초바르고마지막에 큐어크림 바르고메이크업하면 완전 화장잘먹음
리얼코튼은 부드럽게 아기 엉덩이에 터치
피로감도 덜하고 몸이 한결 가벼워진 느낌
꾸준히 사용하고 있는 리파캐럿


In [541]:
total['check'] = total.sentence_form.str.find('OO')
for row in total[total.check > -1].sentence_form:
    print(row)

OO이유식 들어가면서 만들어줘야겠다 생각했는데 역시나 넘나 간편한 베이비쿡솔로
OO때부터 애정하며 사용중이지
OO칫솔 로 핫한 쿨샤전동칫솔 로 바꿨어요
OO가 사용할거라서 가성비 좋은 놈으로 겟
환절기라 메이크업을 해도 늘 좀 칙칙한 느낌이 있었는데 요즘은 화사한 OO랄까
패딩을 입어도 빛이나는 건 OO씨가 예뻐서 그런건가 패딩이 예뻐서 그런건가
OO씨가 이날 장착한 패딩과 신발은 모두 오니즈카타이거 제품이었는데요
거울보니까 내가봐도 예쁜데 OO도 자꾸 예쁘다해주니까 좋다
디올선미레드 라는 애칭이 벌써 붙었을만큼 핫한 000레드 컬러는 OO씨도 사용했네욤
OO추천템
좋아하는 뷰티유튜버 OO가 만든 쿠션제품이라고 해서 어떨지 넘나 궁금했는데 직접 써보니 대박 촉촉하고 컬러감 대박
OO의 애정템 이라하여 써본것이 벌써 몇 개째 쓰고있는지
이젠 고민말고 OO 너도 하나 겟하는 건 어때
푸석푸석 손상이심한 모발때문에 고민이많았는데 친구 추천으로 써본 헤드스파0 OO씨도 쓴다던데 역시 너무 좋네요
워낙 그립감이 좋아 OO가 써보더니 엄마는 손도 못대게 하더군요
출입이 자유로운 지퍼디자인 덕분에 OO가 답답해하지 않구 좋아하네요
친숙한 캐릭터라 OO가 좋아하구 다용도 끈이 있어 어디든 걸수있다는 장점
디자인도 OO한테 잘어울려서 올여름내내 입히겠어요
OO랑 놀러갈라믄 챙겨야될것들이 느무 많아서 고민이었는데 더블보냉가방 하나믄 끝나겠어요
OO와 함께하는 나들이 에는 보냉백 과 쇼퍼백 을 한번에 해결되는 마들린 더블 보온보냉백
OO가 보자마자 좋아해서 엄마는 그것만 봐도 행복했어요
조금만 더워도 땀흘리는 OO에게 완전 딱 맞는 실내복이네요
자연주의 0배 고농축이라 적은양으로도 세탁효과 대박이고 유해물질을 사용하지않아 안전하니 OO전용으로 세탁할때 사용중이에요
트렌드위드미에서 OO가 사용해 인싸템이 되어버린 톤업썬크림
OO향수로도 유명한 일본 향수 브랜드
디자인도 예쁘고 뚜껑은 OO가 갖고 놀기도 하는중
활동적인 OO에게 편하게 입힐수있어 좋으다
OO 애정템이라 참 좋아해서 

## Reformat

In [49]:
### new
entity_property_pair = [
    '본품#가격', '본품#다양성', '본품#디자인', '본품#인지도', '본품#일반', '본품#편의성', '본품#품질',
    '브랜드#가격', '브랜드#디자인', '브랜드#인지도', '브랜드#일반', '브랜드#품질',
    '제품 전체#가격', '제품 전체#다양성', '제품 전체#디자인', '제품 전체#인지도', '제품 전체#일반', '제품 전체#편의성', '제품 전체#품질',
    '패키지/구성품#가격', '패키지/구성품#다양성', '패키지/구성품#디자인', '패키지/구성품#일반', '패키지/구성품#편의성', '패키지/구성품#품질'
]

more_tokens = ['&name&', '&affiliation&', '&social-security-num&', '&tel-num&', '&card-num&', '&bank-account&', '&num&', '&online-account&']

new_tokens = entity_property_pair + more_tokens

tf_id_to_name = ['True', 'False']
tf_name_to_id = {tf_id_to_name[i]: i for i in range(len(tf_id_to_name))}

polarity_id_to_name = ['positive', 'negative', 'neutral']
polarity_name_to_id = {polarity_id_to_name[i]: i for i in range(len(polarity_id_to_name))}

In [50]:
len(entity_property_pair)

25

In [51]:
def reformat(df):
    ep =[]
    p = []
    for index, row in df.iterrows():
        utterance = row.sentence_form
        id = row.id

        for pair in entity_property_pair:
            isPairInOpinion = False
            if pd.isna(utterance):
                break
            for annotation in row.annotation:
                entity_property = annotation[0]
                sentiment = annotation[2]
                if entity_property not in entity_property_pair:
                    print(row.id, entity_property)
                if sentiment not in polarity_id_to_name:
                    print(row.id, entity_property, sentiment)
                    continue
                if entity_property == pair:
                    ep.append([id, utterance, entity_property, tf_name_to_id['True']])
                    p.append([id, utterance, entity_property, sentiment])
                    isPairInOpinion = True
                    break

            if isPairInOpinion is False:
                ep.append([id, utterance, pair, tf_name_to_id['False']])

    return ep, p

In [52]:
def reformat_p_binary(df):
    p_binary = []
    for i, row in df.iterrows():
        row.id, row.sentence_form, row.entity_property, row.sentiment
        for sentiment in polarity_id_to_name:
            if sentiment == row.sentiment:
                p_binary.append([row.id, row.sentence_form, '#'.join([row.entity_property, row.sentiment]), tf_name_to_id['True']])
            else: 
                p_binary.append([row.id, row.sentence_form, '#'.join([row.entity_property, sentiment]), tf_name_to_id['False']])
    return p_binary

In [53]:
ep_train, p_train = reformat(train)
ep_train = pd.DataFrame(ep_train, columns=['id', 'sentence_form', 'entity_property', 'labels'])
p_train = pd.DataFrame(p_train, columns=['id', 'sentence_form', 'entity_property', 'sentiment'])

ep_dev, p_dev = reformat(dev)
ep_dev = pd.DataFrame(ep_dev, columns=['id', 'sentence_form', 'entity_property', 'labels'])
p_dev = pd.DataFrame(p_dev, columns=['id', 'sentence_form', 'entity_property', 'sentiment'])

In [54]:
### more than 1 entitiy in a single row
# condition = train.annotation.apply(lambda x: len(x)) > 1
# print(train.annotation[condition].count())
# print(train.annotation[condition].apply(lambda x: len(x)).sum())
# 489-235

In [55]:
p_binary_train = reformat_p_binary(p_train)
p_binary_train = pd.DataFrame(p_binary_train, columns=['id', 'sentence_form', 'entity_property', 'labels'])

In [56]:
p_binary_train

,id,sentence_form,entity_property,labels
0,nikluge-sa-2022-train-00001,둘쨋날은 미친듯이 밟아봤더니 기어가 헛돌면서 틱틱 소리가 나서 경악,본품#품질#positive,1
1,nikluge-sa-2022-train-00001,둘쨋날은 미친듯이 밟아봤더니 기어가 헛돌면서 틱틱 소리가 나서 경악,본품#품질#negative,0
2,nikluge-sa-2022-train-00001,둘쨋날은 미친듯이 밟아봤더니 기어가 헛돌면서 틱틱 소리가 나서 경악,본품#품질#neutral,1
3,nikluge-sa-2022-train-00002,이거 뭐 삐꾸를 준 거 아냐 불안하고 거금 투자한 게 왜 이래 싶어서 정이 확 떨어...,본품#품질#positive,1
4,nikluge-sa-2022-train-00002,이거 뭐 삐꾸를 준 거 아냐 불안하고 거금 투자한 게 왜 이래 싶어서 정이 확 떨어...,본품#품질#negative,0
...,...,...,...,...
9583,nikluge-sa-2022-train-03000,마지막으로 귀여워서 집어온 모자,제품 전체#디자인#negative,1
9584,nikluge-sa-2022-train-03000,마지막으로 귀여워서 집어온 모자,제품 전체#디자인#neutral,1
9585,nikluge-sa-2022-train-03001,일본유니클로 사랑해,브랜드#일반#positive,0
9586,nikluge-sa-2022-train-03001,일본유니클로 사랑해,브랜드#일반#negative,1


In [57]:
p_binary_dev = reformat_p_binary(p_dev)
p_binary_dev = pd.DataFrame(p_binary_dev, columns=['id', 'sentence_form', 'entity_property', 'labels'])

In [58]:
p_binary_dev

,id,sentence_form,entity_property,labels
0,nikluge-sa-2022-dev-00001,깔끔하게 부직포 포장으로 되어 있어서 그냥 뜨거운 물에 풍덩 넣어놓고 좀 휘젓어주면...,본품#편의성#positive,0
1,nikluge-sa-2022-dev-00001,깔끔하게 부직포 포장으로 되어 있어서 그냥 뜨거운 물에 풍덩 넣어놓고 좀 휘젓어주면...,본품#편의성#negative,1
2,nikluge-sa-2022-dev-00001,깔끔하게 부직포 포장으로 되어 있어서 그냥 뜨거운 물에 풍덩 넣어놓고 좀 휘젓어주면...,본품#편의성#neutral,1
3,nikluge-sa-2022-dev-00002,목욕할 때마다 넣어봤는데 샤워는 자주 해도 목욕은 그렇게 자주가 아님 이것도 약재는...,본품#일반#positive,0
4,nikluge-sa-2022-dev-00002,목욕할 때마다 넣어봤는데 샤워는 자주 해도 목욕은 그렇게 자주가 아님 이것도 약재는...,본품#일반#negative,1
...,...,...,...,...
9001,nikluge-sa-2022-dev-02793,대형으로 샀더니 잘 맞음,제품 전체#일반#negative,1
9002,nikluge-sa-2022-dev-02793,대형으로 샀더니 잘 맞음,제품 전체#일반#neutral,1
9003,nikluge-sa-2022-dev-02794,안경 쓰고 숨 푸악푸악 내쉬어 봤는데 김서림도 거의 없음,제품 전체#품질#positive,0
9004,nikluge-sa-2022-dev-02794,안경 쓰고 숨 푸악푸악 내쉬어 봤는데 김서림도 거의 없음,제품 전체#품질#negative,1


## Counting

In [59]:
print('binary_multi: ', end=''), print(len(ep_train), len(ep_dev), len(p_train), len(p_dev))
print('binary_binary: ', end=''), print(len(ep_train), len(ep_dev), len(p_binary_train), len(p_binary_dev))
ep_train = ep_train.drop_duplicates()
ep_dev = ep_dev.drop_duplicates()
p_train = p_train.drop_duplicates()
p_dev = p_dev.drop_duplicates()
p_binary_train = p_binary_train.drop_duplicates()
p_binary_dev = p_binary_dev.drop_duplicates()
print('\nafter drop_duplicates')
print('binary_multi: ', end=''), print(len(ep_train), len(ep_dev), len(p_train), len(p_dev))
print('binary_binary: ', end=''), print(len(ep_train), len(ep_dev), len(p_binary_train), len(p_binary_dev))

binary_multi: 75000 69825 3196 3002
binary_binary: 75000 69825 9588 9006

after drop_duplicates
binary_multi: 75000 69825 3196 3002
binary_binary: 75000 69825 9588 9006


(None, None)

## Save Files

In [60]:
save_path = './dataset/cleaned_v1'

train.to_csv(f'{save_path}/raw_train.csv', index=False)
dev.to_csv(f'{save_path}/raw_dev.csv', index=False)
test.to_csv(f'{save_path}/raw_test.csv', index=False)

ep_train.to_csv(f'{save_path}/ce_train.csv', index=False)
ep_dev.to_csv(f'{save_path}/ce_dev.csv', index=False)
p_train.to_csv(f'{save_path}/pc_train.csv', index=False)
p_dev.to_csv(f'{save_path}/pc_dev.csv', index=False)
p_binary_train.to_csv(f'{save_path}/pc_binary_train.csv', index=False)
p_binary_dev.to_csv(f'{save_path}/pc_binary_dev.csv', index=False)

# ASC Augmentation

In [ ]:
model_checkpoint = '/content/drive/MyDrive/aspect_based_sentiment_analysis/base_model/klue_roberta_base/v2/klue_roberta_base_mlm/checkpoint-19860'

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)

In [ ]:
sTokens = tokenizer.all_special_tokens

def delTokens(sent):
    sent = sent.split(' ')
    temp = []
    for e in sent:
        if e not in sTokens:
            temp.append(e)
    return ' '.join(temp)

In [ ]:
positive, negative, neutral = p_train[p_train.sentiment == 'positive'], p_train[p_train.sentiment == 'negative'], p_train[p_train.sentiment == 'neutral']

In [ ]:
len(positive), len(negative), len(neutral)

(3043, 58, 95)

In [ ]:
(58 * 3) * 4 * 3, (95 * 3) * 4 * 2 # bt ri rr

(2088, 2280)

Back Translation / Random Insertion / Random Replacement / Random Swap / Random Deletion

In [ ]:
def backTrans(text):
    aug1 = ts.papago(text, sleep_seconds=5, from_language='ko', to_language='en')
    aug1 = ts.papago(aug1, sleep_seconds=5, from_language='en', to_language='ko')

    aug2 = ts.papago(text, sleep_seconds=5, from_language='ko', to_language='ja')
    aug2 = ts.papago(aug2, sleep_seconds=5, from_language='ja', to_language='ko')

    return [aug1, aug2]

def randomInsert(num, sample, device):
    aug = naw.ContextualWordEmbsAug(
        model_path=model_checkpoint, action="insert", model_type='bert', top_k=5, aug_p=0.3, aug_min=1, aug_max=1, device=device)

    aug_result = aug.augment(sample, n=num, num_thread=12)
    aug_result = list(map(delTokens, aug_result))
    aug_result = list(set(aug_result))
    return aug_result

def randomReplace(num, sample, device):
    aug = naw.ContextualWordEmbsAug(
        model_path=model_checkpoint, action="insert", model_type='bert', top_k=5, aug_p=0.3, aug_min=1, aug_max=1, device=device)

    aug_result = aug.augment(sample, n=num, num_thread=12)
    aug_result = list(map(delTokens, aug_result))
    aug_result = list(set(aug_result))
    return aug_result

def randomSwap(num, sample):
    aug = naw.RandomWordAug(action='swap', aug_min=1, aug_max=1, aug_p=0.3)    
    aug_result = aug.augment(sample, n=num, num_thread=2)
    aug_result = list(set(aug_result))
    return aug_result

def randomSplit(num, sample):
    aug = naw.SplitAug(aug_min=1, aug_max=1, aug_p=0.3, min_char=3)
    aug_result = aug.augment(sample, n=num, num_thread=2)
    aug_result = list(set(aug_result))
    return aug_result

In [ ]:
(58 * 3) * 5 * 4, (95 * 3) * 4 * 3 # bt ri rr

(3480, 3420)

In [ ]:
def backtransRoutine(data2augment, output_path):
    print('back translation started.')
    temp = []
    for row in data2augment:
        augs = backTrans(row[1])
        for aug in augs:
            if aug != '' and aug != row[1]:
                new = [row[0], aug, row[2], row[3]]
                if new not in data2augment:
                    temp.append(new)
    data2augment.extend(temp)
    print(len(f'back translation finished.\ncurrent count: {len(data2augment)}'))

    data_aug = pd.DataFrame(data2augment, columns=['id', 'sentence_form', 'entity_property', 'sentiment'])
    data_aug.to_csv(f'{output_path}', index=False)

    return data_aug

In [ ]:
import os

def edaRoutine(data2augment, ri, rr, output_path):
    print(f'current count: {len(data2augment)}')
    print('random insertion started.')
    temp = []
    for row in data2augment:
        augs = randomInsert(ri, row[1], 'cuda')
        for aug in augs:
            if aug != '' and aug != row[1]:
                new = [row[0], aug, row[2], row[3]]
                if new not in data2augment:
                    temp.append(new)
    data2augment.extend(temp)
    print(f'random insertion finished.\ncurrent count: {len(data2augment)}')

    print('random replacement started.')
    temp = []
    for row in data2augment:
        augs = randomReplace(rr, row[1], 'cuda')
        for aug in augs:
            if aug != '' and aug != row[1]:
                new = [row[0], aug, row[2], row[3]]
                if new not in data2augment:
                    temp.append(new)
    data2augment.extend(temp)
    print(f'random replacement finished.\ncurrent count: {len(data2augment)}')

    print('random swap and split started.')
    while len(data2augment) < len(positive):
        temp = []
        k = random.randrange(len(negative))
        id, text, entity, sentiment = data2augment[k]

        selector = random.randint(0,1)
        if selector == 0:
            augs = randomSwap(1, text)
            for aug in augs:
                if aug != '' and aug != text:
                    new = [id, aug, entity, sentiment]
                    if new not in data2augment:
                        temp.append(new)
            data2augment.extend(temp)
        else:
            augs = randomSplit(1, text)
            for aug in augs:
                if aug != '' and aug != text:
                    new = [id, aug, entity, sentiment]
                    if new not in data2augment:
                        temp.append(new)
            data2augment.extend(temp)
        if len(data2augment)%25 == 0:
            print(f'random swap and split in progress.\ncurrent count: {len(data2augment)}')

    print(f'whole augmentation routine finished.\ntotal count: {len(data2augment)}')

    data_aug = pd.DataFrame(data2augment, columns=['id', 'sentence_form', 'entity_property', 'sentiment'])
    data_aug.to_csv(f'{output_path}', index=False)

    return data_aug

In [ ]:
### negative
# # back translation

# data2augment = negative.values.tolist()

output_folder = '/content/drive/MyDrive/aspect_based_sentiment_analysis/data/v11'
output_file = 'negative_bt.csv'
output_path = os.path.join(output_folder, output_file)

# negative_bt = backtransRoutine(data2augment, output_path)
negative_bt = pd.read_csv(output_path)
negative_bt = negative_bt.values.tolist()
# RI / RR

ri = 4 # times - 1
rr = 3 # times - 1

output_folder = '/content/drive/MyDrive/aspect_based_sentiment_analysis/data/v11'
output_file = 'negative_aug.csv'
output_path = os.path.join(output_folder, output_file)

# negative_aug = edaRoutine(negative_bt, ri, rr, output_path)
negative_aug = pd.read_csv(output_path)

In [ ]:
negative_aug
negative_aug = negative_aug.drop_duplicates()

In [ ]:
# negative_aug.sample(n=15, frac=None, replace=False, weights=None, random_state=None, axis=None, ignore_index=False)

In [ ]:
# negative_aug.sort_values('id').head(50).sentence_form

In [ ]:
### neutral
# back translation

# data2augment = neutral.values.tolist()

output_folder = '/content/drive/MyDrive/aspect_based_sentiment_analysis/data/v11'
output_file = 'neutral_bt.csv'
output_path = os.path.join(output_folder, output_file)

# neutral_bt = backtransRoutine(data2augment, output_path)
neutral_bt = pd.read_csv(output_path)
neutral_bt = neutral_bt.values.tolist()

# RI / RR

ri = 3 # times - 1
rr = 2 # times - 1

output_folder = '/content/drive/MyDrive/aspect_based_sentiment_analysis/data/v11'
output_file = 'neutral_aug.csv'
output_path = os.path.join(output_folder, output_file)

# neutral_aug = edaRoutine(neutral_bt, ri, rr, output_path)
neutral_aug = pd.read_csv(output_path)

In [ ]:
neutral_aug
neutral_aug = neutral_aug.drop_duplicates()

In [ ]:
# neutral_aug.sample(n=15, frac=None, replace=False, weights=None, random_state=None, axis=None, ignore_index=False)

In [ ]:
# neutral_aug.sort_values('id').head(50).sentence_form

In [ ]:
p_train_aug = pd.concat([positive, negative_aug, neutral_aug])

In [ ]:
def reformat_p_binary(df):
    p_binary = []
    for i, row in df.iterrows():
        row.id, row.sentence_form, row.entity_property, row.sentiment
        for sentiment in polarity_id_to_name:
            if sentiment == row.sentiment:
                p_binary.append([row.id, row.sentence_form, '#'.join([row.entity_property, row.sentiment]), tf_name_to_id['True']])
            else: 
                p_binary.append([row.id, row.sentence_form, '#'.join([row.entity_property, sentiment]), tf_name_to_id['False']])
    return p_binary

In [ ]:
p_binary_train_aug = reformat_p_binary(p_train_aug)
p_binary_train_aug = pd.DataFrame(p_binary_train_aug, columns=['id', 'sentence_form', 'entity_property', 'labels'])

In [ ]:
p_binary_train_aug

,id,sentence_form,entity_property,labels
0,nikluge-sa-2022-train-00003,간사하게도 그 이후에는 라이딩이 아주 즐거워져서 만족스럽게 탔다.,제품 전체#일반#positive,0
1,nikluge-sa-2022-train-00003,간사하게도 그 이후에는 라이딩이 아주 즐거워져서 만족스럽게 탔다.,제품 전체#일반#negative,1
2,nikluge-sa-2022-train-00003,간사하게도 그 이후에는 라이딩이 아주 즐거워져서 만족스럽게 탔다.,제품 전체#일반#neutral,1
3,nikluge-sa-2022-train-00006,지금 내 실력과 저질 체력으로는 이 정도 자전거도 되게 훌륭한 거라는..,제품 전체#일반#positive,0
4,nikluge-sa-2022-train-00006,지금 내 실력과 저질 체력으로는 이 정도 자전거도 되게 훌륭한 거라는..,제품 전체#일반#negative,1
...,...,...,...,...
27316,nikluge-sa-2022-train-00048,600원짜리 컵인데 더 이상을 바랄 없을 순 듯.,제품 전체#일반#negative,1
27317,nikluge-sa-2022-train-00048,600원짜리 컵인데 더 이상을 바랄 없을 순 듯.,제품 전체#일반#neutral,0
27318,nikluge-sa-2022-train-01352,사진 찍는데 액정으론 내가 괜히샀나 싶 었는데 보정하려고 보니 대만족..,본품#품질#positive,1
27319,nikluge-sa-2022-train-01352,사진 찍는데 액정으론 내가 괜히샀나 싶 었는데 보정하려고 보니 대만족..,본품#품질#negative,1


In [ ]:
p_binary_dev = reformat_p_binary(p_dev)
p_binary_dev = pd.DataFrame(p_binary_dev, columns=['id', 'sentence_form', 'entity_property', 'labels'])

In [ ]:
p_binary_dev

,id,sentence_form,entity_property,labels
0,nikluge-sa-2022-dev-00001,깔끔하게 부직포 포장으로 되어 있어서 그냥 뜨거운 물에 풍덩 넣어놓고 좀 휘젓어주면...,본품#편의성#positive,0
1,nikluge-sa-2022-dev-00001,깔끔하게 부직포 포장으로 되어 있어서 그냥 뜨거운 물에 풍덩 넣어놓고 좀 휘젓어주면...,본품#편의성#negative,1
2,nikluge-sa-2022-dev-00001,깔끔하게 부직포 포장으로 되어 있어서 그냥 뜨거운 물에 풍덩 넣어놓고 좀 휘젓어주면...,본품#편의성#neutral,1
3,nikluge-sa-2022-dev-00002,목욕할 때마다 넣어봤는데(샤워는 자주 해도 목욕은 그렇게 자주가 아님.. 이것도 약...,본품#일반#positive,0
4,nikluge-sa-2022-dev-00002,목욕할 때마다 넣어봤는데(샤워는 자주 해도 목욕은 그렇게 자주가 아님.. 이것도 약...,본품#일반#negative,1
...,...,...,...,...
9001,nikluge-sa-2022-dev-02793,대형으로 샀더니 잘 맞음.,제품 전체#일반#negative,1
9002,nikluge-sa-2022-dev-02793,대형으로 샀더니 잘 맞음.,제품 전체#일반#neutral,1
9003,nikluge-sa-2022-dev-02794,안경 쓰고 숨 푸악푸악 내쉬어 봤는데 김서림도 거의 없음.,제품 전체#품질#positive,0
9004,nikluge-sa-2022-dev-02794,안경 쓰고 숨 푸악푸악 내쉬어 봤는데 김서림도 거의 없음.,제품 전체#품질#negative,1


# Counting

In [ ]:
len(ep_train), len(ep_dev), len(p_train), len(p_dev)

(75000, 69825, 3196, 3002)

In [ ]:
len(ep_train), len(ep_dev), len(p_binary_train_aug), len(p_binary_dev)

(75000, 69825, 27321, 9006)

In [ ]:
ep_train = ep_train.drop_duplicates()
p_binary_train_aug = p_binary_train_aug.drop_duplicates()
ep_dev = ep_dev.drop_duplicates()
p_binary_dev = p_binary_dev.drop_duplicates()
len(ep_train), len(ep_dev), len(p_binary_train_aug), len(p_binary_dev)

(75000, 69825, 27321, 9006)

# Export

In [ ]:
%cd /content/drive/MyDrive/aspect_based_sentiment_analysis/data/v11

# train.to_csv('raw_train.csv', index=False)
# dev.to_csv('raw_dev.csv', index=False)
# test.to_csv('raw_test.csv', index=False)

ep_train.to_csv('ce_train.csv', index=False)
p_binary_train_aug.to_csv('pc_binary_train_aug.csv', index=False)
ep_dev.to_csv('ce_dev.csv', index=False)
p_binary_dev.to_csv('pc_binary_dev.csv', index=False)

/content/drive/MyDrive/aspect_based_sentiment_analysis/data/v11


In [ ]:
# emojis = pd.concat([ep_train.sentence_form, p_train.sentence_form, ep_dev.sentence_form, p_dev.sentence_form], ignore_index=True, verify_integrity=True).to_frame()
# emojis = list(set(demoji.findall(' '.join(emojis.sentence_form.to_list())).keys()))

In [ ]:
df = pd.read_csv('ce_train.csv')
df[df.id == 'nikluge-sa-2022-train-00065']

,id,sentence_form,entity_property,labels
1600,nikluge-sa-2022-train-00065,봉제선이 앞코 쪽에 딱 한군데 있고 맥스나 테니스 클래식 등보다 훨씬 편하다.,본품#가격,1
1601,nikluge-sa-2022-train-00065,봉제선이 앞코 쪽에 딱 한군데 있고 맥스나 테니스 클래식 등보다 훨씬 편하다.,본품#다양성,1
1602,nikluge-sa-2022-train-00065,봉제선이 앞코 쪽에 딱 한군데 있고 맥스나 테니스 클래식 등보다 훨씬 편하다.,본품#디자인,1
1603,nikluge-sa-2022-train-00065,봉제선이 앞코 쪽에 딱 한군데 있고 맥스나 테니스 클래식 등보다 훨씬 편하다.,본품#인지도,1
1604,nikluge-sa-2022-train-00065,봉제선이 앞코 쪽에 딱 한군데 있고 맥스나 테니스 클래식 등보다 훨씬 편하다.,본품#일반,1
1605,nikluge-sa-2022-train-00065,봉제선이 앞코 쪽에 딱 한군데 있고 맥스나 테니스 클래식 등보다 훨씬 편하다.,본품#편의성,1
1606,nikluge-sa-2022-train-00065,봉제선이 앞코 쪽에 딱 한군데 있고 맥스나 테니스 클래식 등보다 훨씬 편하다.,본품#품질,1
1607,nikluge-sa-2022-train-00065,봉제선이 앞코 쪽에 딱 한군데 있고 맥스나 테니스 클래식 등보다 훨씬 편하다.,브랜드#가격,1
1608,nikluge-sa-2022-train-00065,봉제선이 앞코 쪽에 딱 한군데 있고 맥스나 테니스 클래식 등보다 훨씬 편하다.,브랜드#디자인,1
1609,nikluge-sa-2022-train-00065,봉제선이 앞코 쪽에 딱 한군데 있고 맥스나 테니스 클래식 등보다 훨씬 편하다.,브랜드#인지도,1


In [ ]:
df = pd.read_csv('ce_dev.csv')
df

,id,sentence_form,entity_property,labels
0,nikluge-sa-2022-dev-00001,깔끔하게 부직포 포장으로 되어 있어서 그냥 뜨거운 물에 풍덩 넣어놓고 좀 휘젓어주면...,본품#가격,1
1,nikluge-sa-2022-dev-00001,깔끔하게 부직포 포장으로 되어 있어서 그냥 뜨거운 물에 풍덩 넣어놓고 좀 휘젓어주면...,본품#다양성,1
2,nikluge-sa-2022-dev-00001,깔끔하게 부직포 포장으로 되어 있어서 그냥 뜨거운 물에 풍덩 넣어놓고 좀 휘젓어주면...,본품#디자인,1
3,nikluge-sa-2022-dev-00001,깔끔하게 부직포 포장으로 되어 있어서 그냥 뜨거운 물에 풍덩 넣어놓고 좀 휘젓어주면...,본품#인지도,1
4,nikluge-sa-2022-dev-00001,깔끔하게 부직포 포장으로 되어 있어서 그냥 뜨거운 물에 풍덩 넣어놓고 좀 휘젓어주면...,본품#일반,1
...,...,...,...,...
69820,nikluge-sa-2022-dev-02794,안경 쓰고 숨 푸악푸악 내쉬어 봤는데 김서림도 거의 없음.,패키지/구성품#다양성,1
69821,nikluge-sa-2022-dev-02794,안경 쓰고 숨 푸악푸악 내쉬어 봤는데 김서림도 거의 없음.,패키지/구성품#디자인,1
69822,nikluge-sa-2022-dev-02794,안경 쓰고 숨 푸악푸악 내쉬어 봤는데 김서림도 거의 없음.,패키지/구성품#일반,1
69823,nikluge-sa-2022-dev-02794,안경 쓰고 숨 푸악푸악 내쉬어 봤는데 김서림도 거의 없음.,패키지/구성품#편의성,1


In [ ]:
df = pd.read_csv('pc_binary_train_aug.csv')
df

,id,sentence_form,entity_property,labels
0,nikluge-sa-2022-train-00003,간사하게도 그 이후에는 라이딩이 아주 즐거워져서 만족스럽게 탔다.,제품 전체#일반#positive,0
1,nikluge-sa-2022-train-00003,간사하게도 그 이후에는 라이딩이 아주 즐거워져서 만족스럽게 탔다.,제품 전체#일반#negative,1
2,nikluge-sa-2022-train-00003,간사하게도 그 이후에는 라이딩이 아주 즐거워져서 만족스럽게 탔다.,제품 전체#일반#neutral,1
3,nikluge-sa-2022-train-00006,지금 내 실력과 저질 체력으로는 이 정도 자전거도 되게 훌륭한 거라는..,제품 전체#일반#positive,0
4,nikluge-sa-2022-train-00006,지금 내 실력과 저질 체력으로는 이 정도 자전거도 되게 훌륭한 거라는..,제품 전체#일반#negative,1
...,...,...,...,...
27316,nikluge-sa-2022-train-00048,600원짜리 컵인데 더 이상을 바랄 없을 순 듯.,제품 전체#일반#negative,1
27317,nikluge-sa-2022-train-00048,600원짜리 컵인데 더 이상을 바랄 없을 순 듯.,제품 전체#일반#neutral,0
27318,nikluge-sa-2022-train-01352,사진 찍는데 액정으론 내가 괜히샀나 싶 었는데 보정하려고 보니 대만족..,본품#품질#positive,1
27319,nikluge-sa-2022-train-01352,사진 찍는데 액정으론 내가 괜히샀나 싶 었는데 보정하려고 보니 대만족..,본품#품질#negative,1


In [ ]:
df = pd.read_csv('pc_binary_dev.csv')
df

,id,sentence_form,entity_property,labels
0,nikluge-sa-2022-dev-00001,깔끔하게 부직포 포장으로 되어 있어서 그냥 뜨거운 물에 풍덩 넣어놓고 좀 휘젓어주면...,본품#편의성#positive,0
1,nikluge-sa-2022-dev-00001,깔끔하게 부직포 포장으로 되어 있어서 그냥 뜨거운 물에 풍덩 넣어놓고 좀 휘젓어주면...,본품#편의성#negative,1
2,nikluge-sa-2022-dev-00001,깔끔하게 부직포 포장으로 되어 있어서 그냥 뜨거운 물에 풍덩 넣어놓고 좀 휘젓어주면...,본품#편의성#neutral,1
3,nikluge-sa-2022-dev-00002,목욕할 때마다 넣어봤는데(샤워는 자주 해도 목욕은 그렇게 자주가 아님.. 이것도 약...,본품#일반#positive,0
4,nikluge-sa-2022-dev-00002,목욕할 때마다 넣어봤는데(샤워는 자주 해도 목욕은 그렇게 자주가 아님.. 이것도 약...,본품#일반#negative,1
...,...,...,...,...
9001,nikluge-sa-2022-dev-02793,대형으로 샀더니 잘 맞음.,제품 전체#일반#negative,1
9002,nikluge-sa-2022-dev-02793,대형으로 샀더니 잘 맞음.,제품 전체#일반#neutral,1
9003,nikluge-sa-2022-dev-02794,안경 쓰고 숨 푸악푸악 내쉬어 봤는데 김서림도 거의 없음.,제품 전체#품질#positive,0
9004,nikluge-sa-2022-dev-02794,안경 쓰고 숨 푸악푸악 내쉬어 봤는데 김서림도 거의 없음.,제품 전체#품질#negative,1
